# Assignment 3 - Building a Custom Visualization

---

In this assignment you must choose one of the options presented below and submit a visual as well as your source code for peer grading. The details of how you solve the assignment are up to you, although your assignment must use matplotlib so that your peers can evaluate your work. The options differ in challenge level, but there are no grades associated with the challenge level you chose. However, your peers will be asked to ensure you at least met a minimum quality for a given technique in order to pass. Implement the technique fully (or exceed it!) and you should be able to earn full grades for the assignment.


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Ferreira, N., Fisher, D., & Konig, A. C. (2014, April). [Sample-oriented task-driven visualizations: allowing users to make better, more confident decisions.](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;In Proceedings of the SIGCHI Conference on Human Factors in Computing Systems (pp. 571-580). ACM. ([video](https://www.youtube.com/watch?v=BI7GAs-va-Q))


In this [paper](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) the authors describe the challenges users face when trying to make judgements about probabilistic data generated through samples. As an example, they look at a bar chart of four years of data (replicated below in Figure 1). Each year has a y-axis value, which is derived from a sample of a larger dataset. For instance, the first value might be the number votes in a given district or riding for 1992, with the average being around 33,000. On top of this is plotted the 95% confidence interval for the mean (see the boxplot lectures for more information, and the yerr parameter of barcharts).

<br>
<img src="Assignment3Fig1.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Figure 1 from (Ferreira et al, 2014).</h4>

<br>

A challenge that users face is that, for a given y-axis value (e.g. 42,000), it is difficult to know which x-axis values are most likely to be representative, because the confidence levels overlap and their distributions are different (the lengths of the confidence interval bars are unequal). One of the solutions the authors propose for this problem (Figure 2c) is to allow users to indicate the y-axis value of interest (e.g. 42,000) and then draw a horizontal line and color bars based on this value. So bars might be colored red if they are definitely above this value (given the confidence interval), blue if they are definitely below this value, or white if they contain this value.


<br>
<img src="Assignment3Fig2c.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  Figure 2c from (Ferreira et al. 2014). Note that the colorbar legend at the bottom as well as the arrows are not required in the assignment descriptions below.</h4>

<br>
<br>

**Easiest option:** Implement the bar coloring as described above - a color scale with only three colors, (e.g. blue, white, and red). Assume the user provides the y axis value of interest as a parameter or variable.


**Harder option:** Implement the bar coloring as described in the paper, where the color of the bar is actually based on the amount of data covered (e.g. a gradient ranging from dark blue for the distribution being certainly below this y-axis, to white if the value is certainly contained, to dark red if the value is certainly not contained as the distribution is above the axis).

**Even Harder option:** Add interactivity to the above, which allows the user to click on the y axis to set the value of interest. The bar colors should change with respect to what value the user has selected.

**Hardest option:** Allow the user to interactively set a range of y values they are interested in, and recolor based on this (e.g. a y-axis band, see the paper for more details).

---

*Note: The data given for this assignment is not the same as the data used in the article and as a result the visualizations may look a little different.*

In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import scipy.stats as stats

In [ ]:
COLUMN = [1992, 1993, 1994, 1995]

COLOR_LIST = ['MidnightBlue', 'MediumBlue', 'CornflowerBlue', 'LightBlue', 
             'AliceBlue', 'White', 'MistyRose', 'LightSalmon', 'IndianRed', 
             'FireBrick', 'DarkRed']
BOUNDS     = [0.00, 0.09, 0.18, 0.27, 0.36, 0.45, 
              0.55, 0.64, 0.73, 0.82, 0.91, 1.00]

BAR_DIST   ={"norm0" : {"mu" : 32000, "sigma" : 200000}, 
             "norm1" : {"mu" : 43000, "sigma" : 100000}, 
             "norm2" : {"mu" : 43500, "sigma" : 140000}, 
             "norm3" : {"mu" : 48000, "sigma" : 70000}}

BAR_TITLES = ["N(32,200)", "N(43,100)", "N(43.5,140)", "N(48,70)", ]


# DISTRIBUTION_DATA = 

The Central Limit Theorem for Sample Means:

X¯∼N(μx,σx/√n)
 
The Mean  X¯:σx 
Central Limit Theorem for Sample Means z-score and standard error of the mean:
t=(x_bar − μ_x)/(σ_x/√n)
 
Standard Error of the Mean (Standard Deviation ( X¯ )):
    σ_x/√n

In [ ]:
def getConfidenceInterval(summary, confidence, srcData) :
    
    confidenceInterval = []

    degFreedom = summary.loc['count'][1992] - 1
    alpha = (1 - confidence)/2.0
    
    maximumBar = summary.loc['min'].min()
  
    for i in COLUMN :
        tValue = stats.t.ppf(q = alpha, df = degFreedom)
        if tValue < 0.0 :
            tValue = -tValue
        
        ci = tValue*summary.loc['std'][i]/math.sqrt(summary.loc['count'][i])
        confidenceInterval.append(ci)

        barHeight = summary.loc['mean'][i] + ci
        if barHeight > maximumBar :
            maximumBar = barHeight

    idx = len(summary)
    summary.loc[idx] = confidenceInterval
    summary.rename(index={idx:'CI'},inplace=True)

    return maximumBar



In [ ]:
def getGraphLimits(srcData, nticks, confidence) :
    summary = srcData[COLUMN].describe()

    maximumBar = getConfidenceInterval(summary, confidence, srcData)
    print("summary = ", summary)

    yTickLength = maximumBar/nticks
    n = int(np.log10(yTickLength))
    yTickLength = int(yTickLength / (10**n)) * 10**n
    ticks = math.ceil(maximumBar/yTickLength)

    return [summary, yTickLength, ticks]

In [ ]:
def getColorbar(ax) :
    cmap = mpl.colors.ListedColormap(COLOR_LIST)
    norm = mpl.colors.BoundaryNorm(BOUNDS, cmap.N)

#     plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("bottom", size="5%", pad=0.5)

    cb=plt.colorbar(mpl.cm.ScalarMappable(norm, cmap), cax = cax, orientation='horizontal')
    cb.outline.set_edgecolor('black')


In [ ]:
def getBarColors(ax, summary, randDataT, threshold) :
    # get two-tailed, one sample t-test
    # select color based on the result

    colorBar = []
    pvalue = []
    statistic = []
    for i in COLUMN :
        pvalue.append(0)
        statistic.append(0)
        colorBar.append(int(0))

    for i in range(len(COLUMN)) :
        # computed t value
        result = stats.ttest_1samp(randDataT[COLUMN[i]], threshold, alternative = 'two-sided')
        pvalue[i] = result.pvalue/2.
        statistic[i] = result.statistic
        print("statistic = ", result.statistic, " pvalue = ", pvalue[i])

        if summary.loc['mean'][COLUMN[i]] > threshold :
            pvalue[i] = 1 - result.pvalue
        print("statistic = ", result.statistic, " pvalue = ", pvalue[i])

#         if pvalue[i] >= 1 :
#             colorBar[i] = len(BOUNDS) - 1
#             break
#         elif pvalue[i] <= 0 :
#             colorBar[i] = 0
#             break
#         else: 
        for j in range(len(BOUNDS)) :
            print("pvalue[i] ", pvalue[i], " BOUNDS[j] = ", BOUNDS[j])
            if pvalue[i]  <= BOUNDS[j] :
                colorBar[i] = j-1
                if colorBar[i] < 0 :
                    colorBar[i] = 0
                break

    print("colorBar = ", colorBar)
    idx = len(summary)

    summary.loc[idx] = colorBar
    summary.rename(index={idx:'color'},inplace=True)
    summary.loc[idx] = pvalue
    summary.rename(index={idx:'pvalue'},inplace=True)
    summary.loc[idx] = statistic
    summary.rename(index={idx:'statistic'},inplace=True)
    print(summary)

            

In [ ]:
def graph(randDataT, limits, threshold) :
    summary = limits[0]
    yTickLength = limits[1]
    ticks = limits[2]

    fig = plt.figure(num=1, figsize=(6, 6))
    ax  = fig.add_axes([0,0,1,1])
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    getBarColors(ax, summary, randDataT, threshold)

#     y_offset = np.zeros(len(COLUMN))
    x_pos = [i for i, _ in enumerate(COLUMN)]
    lStyle    = '-'

#     print('Data type of each column of Dataframe :')
#     print(summary.dtypes)
#     print("len(COLUMN) = ", len(COLUMN))

    for col in range(len(COLUMN)) :
#         print("col = ", col, " ", COLUMN[col], " ", x_pos)
#         print("COLOR_LIST[int(summary.loc['color'][COLUMN[col]])] = ", COLOR_LIST[int(summary.loc['color'][COLUMN[col]])])
#         print("int(summary.loc['color'][COLUMN[col]]) = ", int(summary.loc['color'][COLUMN[col]]))
#         print("summary.loc['color'] = ", summary.loc['color'])
#         print("COLUMN[col] = ", COLUMN[col])

        plt.bar(x_pos[col], 
                summary.loc['mean'][COLUMN[col]],  
                yerr = summary.loc['CI'][COLUMN[col]], 
                edgecolor ='black',
                linestyle = lStyle,
                color=COLOR_LIST[int(summary.loc['color'][COLUMN[col]])],
                align='center',
                capsize=10,
                width=1.0)

    plt.xticks(x_pos, BAR_TITLES)
    plt.xlabel("Distribution")
    plt.ylabel("Sample Mean and Error")
    plt.title("Comparison of Normal Distributions\nwith Different mu and sigma parameters")

    ax.plot([-1.0, 4.5], [threshold, threshold], "k--")

    getColorbar(ax)



In [ ]:
# Use the following data for this assignment:

# import pandas as pd
# import numpy as np

def distributionBarChart(ntick = 10, threshold = 0, confidence = 0.95) :
    np.random.seed(12345)

    randData = pd.DataFrame([np.random.normal(32000,200000,3650), 
                       np.random.normal(43000,100000,3650), 
                       np.random.normal(43500,140000,3650), 
                       np.random.normal(48000,70000,3650)], 
                      index=COLUMN)
#     print(randData)
    randDataT = randData.T
    nticks = 10
    limits = [summary, yTickLength, ticks] = getGraphLimits(randDataT, nticks, confidence)

    graph(randDataT, limits, threshold)
    print(summary)
    plt.show()

#     randDataT

In [ ]:
distributionBarChart(10, 33000.0, 0.95)

In [ ]:
import math
x = 463.6578
z = float(math.ceil(x / 100.0) * 100)
print(type(z))
print(z)

In [ ]:
200000/math.sqrt(3600)

In [ ]:
stats.t.ppf(1-.95/2, 3600)

In [ ]:
## 45678911234567892123456789312345678941234567895123456789612345678971234567898


In [30]:
## #############################################################################
## Kevin Glass
## Applied Plotting, Charting & Data Representation in Python
## Assignment 3
## #############################################################################

import numpy as np
import pandas as pd
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import scipy.stats as stats


## #############################################################################
## The following lists are used as constants to ensure consist names and values
## 
## COLUMN references the rows of the randData or the columns of the transposed 
## COLOR_LIST is the ordered list of color
## BOUNDS defines the end points of each color category
## BAR_TITLES I didn't like the 1992-1995 because they are really just 
##            distributions, so I named the bars after their associated 
##            distribution.
## BAR_DIST is for future use
## 
## #############################################################################
COLUMN = [1992, 1993, 1994, 1995]
COLOR_LIST = ['MidnightBlue', 'MediumBlue', 'CornflowerBlue', 'LightBlue', 
             'AliceBlue', 'White', 'MistyRose', 'LightSalmon', 'IndianRed', 
             'FireBrick', 'DarkRed']
BOUNDS     = [0.00, 0.09, 0.18, 0.27, 0.36, 0.45, 
              0.55, 0.64, 0.73, 0.82, 0.91, 1.00]
BAR_DIST   ={"norm0" : {"mu" : 32000, "sigma" : 200000}, 
             "norm1" : {"mu" : 43000, "sigma" : 100000}, 
             "norm2" : {"mu" : 43500, "sigma" : 140000}, 
             "norm3" : {"mu" : 48000, "sigma" : 70000}}
BAR_TITLES = ["N(32,200)", "N(43,100)", "N(43.5,140)", "N(48,70)", ]


## #############################################################################
## 
## The confidence interval is computed using a double-sided single-sample t-test
## and computing the standard error using summary data from the source data. 
## Incidentally, it was convenient to also compute the highest error bar,which 
## was used to set up the height of the frame.
## 
## #############################################################################

def getConfidenceInterval(summary, confidence, srcData) :
    
    confidenceInterval = []

    degFreedom = summary.loc['count'][1992] - 1
    alpha = (1 - confidence)/2.0
    
    maximumBar = summary.loc['min'].min()

    for i in COLUMN :
        tValue = stats.t.ppf(q = alpha, df = degFreedom)
        if tValue < 0.0 :
            tValue = -tValue
        
        ci = tValue*summary.loc['std'][i]/math.sqrt(summary.loc['count'][i])
        confidenceInterval.append(ci)

        barHeight = summary.loc['mean'][i] + ci
        if barHeight > maximumBar :
            maximumBar = barHeight

    idx = len(summary)
    summary.loc[idx] = confidenceInterval
    summary.rename(index={idx:'CI'},inplace=True)

    return maximumBar

## #############################################################################
## 
## The summary data is used to computer the number of ticks and the distance 
## between tick marks. IT is also used to set the x- and y- axis limits.
## 
## #############################################################################

def getGraphLimits(srcData, nticks, confidence) :
    summary = srcData[COLUMN].describe()

    ## maximumBar is the largest y value used to set the height of the y-axis
    maximumBar = getConfidenceInterval(summary, confidence, srcData)

    yTickLength = maximumBar/nticks
    n = int(np.log10(yTickLength))
    yTickLength = int(yTickLength / (10**n)) * 10**n
    ticks = math.ceil(maximumBar/yTickLength)

    return [summary, yTickLength, ticks]

## #############################################################################
## The color bar is set by storing the COLOR_LIST in a color map and normalized
## using a BoundaryNorm object. The placement of the color bar was created using
## an axis divider.
## 
## 
## #############################################################################

def getColorbar(ax) :
    cmap = mpl.colors.ListedColormap(COLOR_LIST)
    norm = mpl.colors.BoundaryNorm(BOUNDS, cmap.N)

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("bottom", size="5%", pad=0.5)

    cb=plt.colorbar(mpl.cm.ScalarMappable(norm, cmap), cax = cax, orientation='horizontal')
    cb.outline.set_edgecolor('black')

## #############################################################################
## 
## The color of each bar in the bar graph was determined by computing the double
## sided t-test for a single sample compared to the threshold value supplied by
## the user. The probability value generated by the ttest_1samp function was 
## adjusted to represent both tails of the probability, which was mapped into 
## the color map to define the color used for that bar.
## 
## In addition to mapping colors to bars, the values for the bar color, pvalue 
## and the t-statistic were stored in the summary data dataframe.
## 
## 
## #############################################################################

def getBarColors(ax, summary, randDataT, threshold) :

    colorBar = []
    pvalue = []
    statistic = []
    for i in COLUMN :
        pvalue.append(0)
        statistic.append(0)
        colorBar.append(int(0))

    for i in range(len(COLUMN)) :
        # computed t value
        result = stats.ttest_1samp(randDataT[COLUMN[i]], threshold, alternative = 'two-sided')
        pvalue[i] = result.pvalue/2.
        statistic[i] = result.statistic

        if summary.loc['mean'][COLUMN[i]] > threshold :
            pvalue[i] = 1 - result.pvalue

        for j in range(len(BOUNDS)) :
            if pvalue[i]  <= BOUNDS[j] :
                colorBar[i] = j-1
                if colorBar[i] < 0 :
                    colorBar[i] = 0
                break

    idx = len(summary)

    summary.loc[idx] = colorBar
    summary.rename(index={idx:'color'},inplace=True)
    summary.loc[idx] = pvalue
    summary.rename(index={idx:'pvalue'},inplace=True)
    summary.loc[idx] = statistic
    summary.rename(index={idx:'statistic'},inplace=True)

    print(summary)


## #############################################################################
## This function defines the graph components include the bar characteristics,
## the graph framing, axes, and the graph and axes data.
## 
## #############################################################################

def graph(randDataT, limits, threshold) :
    summary = limits[0]
    yTickLength = limits[1]
    ticks = limits[2]

    fig = plt.figure(num=1, figsize=(15, 8))
    ax  = fig.add_axes([0,0,1,1])
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    getBarColors(ax, summary, randDataT, threshold)

    x_pos = [i for i, _ in enumerate(COLUMN)]
    lStyle    = '-'

    for col in range(len(COLUMN)) :
        plt.bar(x_pos[col], 
                summary.loc['mean'][COLUMN[col]],  
                yerr = summary.loc['CI'][COLUMN[col]], 
                edgecolor ='black',
                linestyle = lStyle,
                color=COLOR_LIST[int(summary.loc['color'][COLUMN[col]])],
                align='center',
                capsize=10,
                width=1.0)

    plt.xticks(x_pos, BAR_TITLES)
    plt.xlabel("Distribution")
    plt.ylabel("Sample Mean and Error")
    plt.title("Comparison of Normal Distributions\nwith Different mu and sigma parameters")

    ax.plot([-1.0, 4.5], [threshold, threshold], "k--")

    getColorbar(ax)


## #############################################################################
## This is the main function. It doesn't do much other than coordinate the other
## functions and it prints the graph.
## 
## #############################################################################

def distributionBarChart(ntick = 10, threshold = 0, confidence = 0.95) :
    np.random.seed(12345)

    randData = pd.DataFrame([np.random.normal(32000,200000,3650), 
                       np.random.normal(43000,100000,3650), 
                       np.random.normal(43500,140000,3650), 
                       np.random.normal(48000,70000,3650)], 
                      index=COLUMN)

    randDataT = randData.T
    nticks = 10
    limits = [summary, yTickLength, ticks] = getGraphLimits(randDataT, nticks, confidence)

    graph(randDataT, limits, threshold)
    plt.savefig('distributions.png', bbox_inches = 'tight')
    plt.show()
   
    

In [ ]:
distributionBarChart(10, 41000.0, 0.95)